In [ ]:
import requests

resps = []
for i in range(0, 200):
    resp = requests.get(f'http://117.186.102.78:32497/api/get_enemy_data?index={i}')
    resps.append(resp)


[0, 93, 124, 117, 18, 17, 75, 15, 11, 24, 0, 12, 9, 17, 10, 5, 8, 4, 13, 9, 5, 7, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
jsons = [resp.json() for resp in resps]
import json
json.dump(jsons, open('enemy_data.json', 'w'), indent=4)

In [16]:

c = [0 for i in range(100)]
for (i, resp) in enumerate(resps):
    for monster in resp.json()['value']['labels']:
        c[len(monster['spawnTimes'])] += 1
        if 'SKILL_627CD8130' in (monster['skills']):
            print(i, monster['skills'])
        # if 'SKILL_E386ECBE0' in (monster['skills']):
        #     print(i, monster['skills'])
# for i in range(1, 50):
#     if c[i]:
#         print(f"{i}: {c[i]}")

1 ['SKILL_627CD8130', 'SKILL_4392456DE']
9 ['SKILL_C17BE3111', 'SKILL_627CD8130']
18 ['SKILL_BD8F56413', 'SKILL_627CD8130']
19 ['SKILL_627CD8130', 'SKILL_16C031199']
21 ['SKILL_627CD8130']
47 ['SKILL_627CD8130']
49 ['SKILL_C17BE3111', 'SKILL_627CD8130']
52 ['SKILL_3A3B1799D', 'SKILL_627CD8130']
58 ['SKILL_627CD8130', 'SKILL_B18C26797']
61 ['SKILL_627CD8130', 'SKILL_4392456DE']
62 ['SKILL_16C031199', 'SKILL_627CD8130', 'SKILL_BD8F56413']
63 ['SKILL_89A8DCA03', 'SKILL_627CD8130', 'SKILL_4392456DE']
73 ['SKILL_627CD8130', 'SKILL_4392456DE']
82 ['SKILL_627CD8130', 'SKILL_89A8DCA03']
95 ['SKILL_4392456DE', 'SKILL_627CD8130']
113 ['SKILL_B18C26797', 'SKILL_627CD8130']
115 ['SKILL_3BC8960A9', 'SKILL_627CD8130']
115 ['SKILL_3BC8960A9', 'SKILL_BD8F56413', 'SKILL_627CD8130']
119 ['SKILL_627CD8130']
120 ['SKILL_627CD8130', 'SKILL_BD8F56413']
125 ['SKILL_627CD8130']
135 ['SKILL_627CD8130', 'SKILL_3A3B1799D']
152 ['SKILL_B18C26797', 'SKILL_627CD8130']
158 ['SKILL_627CD8130', 'SKILL_4392456DE']
162 